## Restricted Botlzman Machines

En la sesión de hoy vamos a implementar una máquina restringida de Botlzman para implementar un sistema de recomendación para datos binarios. Para ello vamos a tratar un dataset de recomendaciones de películas.

El primer paso será cargar las librerías que vamos a usar en este ejercicio.

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.parallel
import torch.utils.data
import os
from sklearn.model_selection import train_test_split 

Lo siguiente será cargar los propios datos. EL primer dataset serán las propias votaciones de los usuarios a distintas películas. Imprime por pantalla las primeras líneas para ver la estructura de los datos.

In [4]:
df_ratings=pd.read_csv("ejercicio-clase-2-ratings.csv")
df_ratings = df_ratings[['userId', 'movieId', 'rating']]
print(df_ratings.head())

   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0


El formato de este dataframe es de la forma usuario-película-puntuación. Lo vamos a pasar a formato matriz en el que tengamos una fila por usuario y las columnas sean las películas.

In [5]:
df_input=pd.pivot_table(df_ratings, values='rating',index='userId',columns='movieId',aggfunc=np.sum)

C:\Users\Marti\AppData\Local\Temp\ipykernel_256\612626875.py:1: FutureWarning: The provided callable <function sum at 0x000001AF6B4087C0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_input=pd.pivot_table(df_ratings, values='rating',index='userId',columns='movieId',aggfunc=np.sum)


Imprime las primeras filas para ver la estructura de la matriz resultante.

In [6]:
df_input.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rellena con -1 los valores faltantes (así se puede detectar fácilmente las películas que un usuario no haya votado).

In [7]:
df_input = df_input.fillna(-1)

Divide el dataset en un subset de entranamiento y uno de evaluación con el 80% y el 20%, respectivamente, de los usuarios.

In [8]:
X_train,X_test = train_test_split(df_input, test_size=0.2, random_state=42)

Los pasamos a tensores de Pytorch

In [9]:
training_set=X_train.values
test_set=X_test.values
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

Vamos a plantear una maquina restringida que trata con valores binarios. Por tanto pasaremos las matrices a valores 0-1 (únicamente los valores 'votados') Por tanto, pasamos las votaciones a valores binarios. Asignaremos el valor 1 a las películas que le hayan gustado al usuario, 0 que no y mantendremos como -1 a las que no ha visto. 

Se propone que las votaciones iguales o por encima de 2.5 se codifiquen como 1 y las que estén por debajo se codifiquen como 0, teniendo en cuenta que hay valores con valores -1.

In [10]:
training_set[((training_set >= 0) & (training_set <= 2))] = 0
training_set[training_set > 2] = 1
test_set[((test_set >= 0) & (test_set <= 2))] = 0
test_set[test_set >= 2] = 1

Defina la clase refelativa a un RBM.

In [17]:
class RBM_2(object):
    def __init__(self, nv, nh, batch_size, epochs, steps, lr, verbose):
        self.w = torch.randn(nh, nv)
        self.bh = torch.randn(1, nh)
        self.bv = torch.randn(1, nv)
        self.nv = nv
        self.nh = nh
        self.batch_size = batch_size
        self.epochs = epochs
        self.steps = steps
        self.lr = lr
        self.verbose = verbose

    def cond_prob_h(self, x):
        wx = torch.mm(x, self.w.t())
        activation = wx + self.bh.expand_as(wx)
        p_h_subject_v = torch.sigmoid(activation)
        return p_h_subject_v, torch.bernoulli(p_h_subject_v)
    
    def cond_prob_v(self, y):
        wy = torch.mm(y, self.w)
        activation = wy + self.bv.expand_as(wy)
        p_v_subject_h = torch.sigmoid(activation)
        return p_v_subject_h, torch.bernoulli(p_v_subject_h)
    
    def update(self, v0, vk, ph0, phk):
        lr = self.lr
        self.w += lr * (torch.t(torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)))
        self.bh += lr * (torch.t(torch.sum((ph0 - phk), 0)))
        self.bv += lr * (torch.t(torch.sum((v0 - vk), 0)))

    def train(self, df_train):
        batch_size = self.batch_size
        epochs = self.epochs
        steps = self.steps
        verbose = self.verbose
        for epoch in range(1, epochs + 1):
            train_loss = 0
            s = 0.
            users = len(df_train)
            for user in range(0, users - batch_size, batch_size):
                vk = df_train[user:user + batch_size]
                v0 = df_train[user:user + batch_size]
                ph0,_ = self.cond_prob_h(v0)
                for k in range(steps):
                    _,hk = self.cond_prob_h(vk)
                    _,vk = self.cond_prob_v(hk)
                    vk[v0<0] = v0[v0<0]
                phk,_ = self.cond_prob_h(vk)
                self.update(v0, vk, ph0, phk)
                train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
                s += 1.
            if verbose:
                print('Epoch {epoch} loss: {loss}'.format(epoch=epoch, loss=train_loss/s))

    def evlauate(self, df_test):
        verbose = self.verbose
        users = len(df_test)
        test_loss = 0
        s = 0
        for user in range(users):
            v = df_test[user:user + 1]
            vt = df_test[user:user + 1]
            if len(vt[vt>=0]) > 0:
                _,h = self.cond_prob_h(v)
                _,v = self.cond_prob_v(h)
                test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
                s += 1.
        if verbose:
            print('Test loss: {loss}'.format(loss=test_loss/s))

    def predict(self, v):
        _,h = self.cond_prob_h(v)
        probs, v = self.cond_prob_v(h)
        return probs, v       
        

Define una RBM con tantas neuronas como el número de películas que haya definidas. Pruebe con 100 neuronas ocultas, 30 épocas y un tamaño de lote de 100. Entrene su RBM a continuación.

In [18]:
rbm = RBM_2(nv = len(training_set[0]), nh = 100, batch_size = 100, epochs = 30, steps = 10, lr = 0.5, verbose = True)
rbm.train(training_set)

Epoch 1 loss: 0.38880226016044617
Epoch 2 loss: 0.2226582020521164
Epoch 3 loss: 0.21822503209114075
Epoch 4 loss: 0.1993342787027359
Epoch 5 loss: 0.18877331912517548
Epoch 6 loss: 0.18536008894443512
Epoch 7 loss: 0.17867016792297363
Epoch 8 loss: 0.17767685651779175
Epoch 9 loss: 0.17462903261184692
Epoch 10 loss: 0.18178609013557434
Epoch 11 loss: 0.17739242315292358
Epoch 12 loss: 0.17306289076805115
Epoch 13 loss: 0.17275197803974152
Epoch 14 loss: 0.17321474850177765
Epoch 15 loss: 0.17121148109436035
Epoch 16 loss: 0.17266595363616943
Epoch 17 loss: 0.16924752295017242
Epoch 18 loss: 0.17016425728797913
Epoch 19 loss: 0.17169061303138733
Epoch 20 loss: 0.16845490038394928
Epoch 21 loss: 0.16633979976177216
Epoch 22 loss: 0.1673937290906906
Epoch 23 loss: 0.16746780276298523
Epoch 24 loss: 0.16696599125862122
Epoch 25 loss: 0.16626699268817902
Epoch 26 loss: 0.1655222326517105
Epoch 27 loss: 0.16443118453025818
Epoch 28 loss: 0.16372527182102203
Epoch 29 loss: 0.1634774506092071

Evalua cómo se comporta el modelo ante los datos no usados en el entrenamiento.

In [19]:
rbm.evlauate(test_set)

Test loss: 0.20331263542175293


Seleccione un usuario del set de entrenamiento y determine las películas entre las que no ha visto, las que se considerará con mayor probabilidad que le gusten.

In [43]:
import random
r1 = random.randint(0, test_set.shape[0])
tmp_user = test_set[r1:r1+1]
probs, pred = rbm.predict(tmp_user)
likelihood = tmp_user * pred * probs
print("Most probable: ", torch.argmin(likelihood))

Most probable:  tensor(6298)
